In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-12-26"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
10261,2024-12-26,Eua Nba,00:30,Phoenix Suns,Denver Nuggets,2.20,1.72,231.5,1.84,2.00,1.5,2.02,1.83,jmtrJC4b,0.454545,0.581395,0.543478,0.500000,0.035941,0.2,1.0,NaN,NaN,189.186,35.903507,0.189779,185.826,45.585394,0.245312,155.892,210.624,154.29,157.08,0.0,0.0,0.0,0.0,0.173169,0.058926,0.069792,-0.74,-0.148,-8.108108,0.666763,0.6,-0.066763,0.68,0.136,5.294118,0.598495,0.6,0.001505
10262,2024-12-26,Eua Nba,21:00,Indiana Pacers,Oklahoma City Thunder,2.69,1.39,225.5,1.82,1.99,4.5,2.05,1.77,dpqjHjZA,0.371747,0.719424,0.549451,0.502513,0.091172,0.4,0.4,NaN,NaN,270.142,114.996726,0.425690,145.698,12.085354,0.082948,148.742,168.746,171.36,148.05,0.0,0.0,0.0,0.0,0.450607,0.063101,0.103660,-1.02,-0.204,-8.284314,0.687215,0.7,0.012785,0.57,0.114,3.421053,0.683402,0.7,0.016598
10263,2024-12-26,Eua Nba,21:00,Orlando Magic,Miami Heat,1.85,2.02,208.5,1.80,1.95,-2.5,2.00,1.77,GYeObrJq,0.540541,0.495050,0.555556,0.512821,0.035590,0.4,0.6,NaN,NaN,268.176,104.613113,0.390091,176.904,52.288112,0.295573,264.206,228.774,428.76,187.24,1.0,0.0,0.0,0.0,0.062123,0.056569,0.086278,2.07,0.414,2.053140,0.559068,0.8,0.240932,-3.39,-0.678,-1.504425,0.525852,0.4,-0.125852
10264,2024-12-26,Eua Nba,21:00,Washington Wizards,Charlotte Hornets,2.04,1.66,225.5,1.82,1.99,1.5,2.03,1.79,ObaWdMmd,0.490196,0.602410,0.549451,0.502513,0.092606,0.8,0.6,NaN,NaN,889.372,505.762930,0.568674,341.108,188.552647,0.552765,600.796,456.590,302.58,181.26,0.0,0.0,0.0,0.0,0.145244,0.063101,0.088851,4.96,0.992,1.048387,0.232533,0.2,-0.032533,0.89,0.178,3.707865,0.338311,0.1,-0.238311
10265,2024-12-26,Eua Nba,21:30,Atlanta Hawks,Chicago Bulls,1.32,2.99,242.5,1.82,1.99,-5.5,1.91,1.95,v7sJKPu3,0.757576,0.334448,0.549451,0.502513,0.092024,0.6,0.4,NaN,NaN,240.324,61.832248,0.257287,342.558,336.424383,0.982095,239.940,411.548,293.67,941.85,0.0,1.0,0.0,0.0,0.547967,0.063101,0.014655,1.85,0.370,0.864865,0.487554,0.4,-0.087554,8.71,1.742,1.142365,0.372201,0.7,0.327799
10266,2024-12-26,Eua Nba,22:00,Memphis Grizzlies,Toronto Raptors,1.10,5.43,236.5,1.84,1.96,-12.5,2.04,1.78,6sNrxGnA,0.909091,0.184162,0.543478,0.510204,0.093253,0.6,0.4,NaN,NaN,179.268,34.367908,0.191712,434.046,315.391131,0.726631,181.484,470.436,158.40,937.50,0.0,1.0,0.0,0.0,0.937756,0.044659,0.096255,0.65,0.130,0.769231,0.702852,0.8,0.097148,-2.90,-0.580,-7.637931,0.256467,0.1,-0.156467
10267,2024-12-26,Eua Nba,22:00,Milwaukee Bucks,Brooklyn Nets,1.20,3.84,215.5,1.82,1.99,-9.5,1.97,1.83,CKuRI3AF,0.833333,0.260417,0.549451,0.502513,0.093750,0.6,0.4,NaN,NaN,217.312,85.488574,0.393391,447.522,162.418262,0.362928,161.110,429.712,161.20,455.05,0.0,0.0,0.0,0.0,0.740779,0.063101,0.052103,0.15,0.030,6.666667,0.753207,0.7,-0.053207,1.25,0.250,11.360000,0.248513,0.4,0.151487
10268,2024-12-26,Eua Nba,22:00,New Orleans Pelicans,Houston Rockets,3.65,1.22,221.5,1.82,1.99,7.5,2.05,1.77,IXARGsuS,0.273973,0.819672,0.549451,0.502513,0.093645,0.6,0.6,NaN,NaN,386.220,114.276822,0.295885,183.442,37.803361,0.206078,320.274,186.858,508.26,158.46,1.0,0.0,0.0,0.0,0.705655,0.063101,0.103660,-2.94,-0.588,-4.506803,0.476652,0.3,-0.176652,-0.26,-0.052,-4.230769,0.538293,0.7,0.161707
10269,2024-12-26,Europa Euroliga,15:00,Alba Berlin,Real Madrid,5.40,1.18,171.5,1.91,1.91,9.5,2.00,1.71,OfDRfsqK,0.185185,0.847458,0.523560,0.523560,0.032643,0.6,0.4,NaN,NaN,482.382,381.130825,0.790102,130.118,35.840343,0.275445,238.392,142.332,195.36,183.36,0.0,0.0,0.0,0.0,0.906988,0.000000,0.110545,-1.2

## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,00:30,Eua Nba,Phoenix Suns,Denver Nuggets,231.5,1.84,0.8770,Over
1,21:30,Eua Nba,Atlanta Hawks,Chicago Bulls,242.5,1.82,0.8121,Over
